In [3]:
import numpy as np
import copy

In [8]:
class PipingSystem:
    
    def __init__(self, file_path):
        self.file_path = file_path
        self.mean_points = ['*','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
        self.operation_system_active = None
        
    def GetSystemList(self):
        #getting pipe system information
        with open(self.file_path, 'r') as file:
            system_s = file.read().split("\n")
        file.close()
        
        system_s = [item.replace(" ", ",") for item in system_s]
        system = []
        for item in system_s:
            item = item.split(",")
            item[1] = list(map(int, item[1:3]))
            del item[2]
            system.append(item)
        print(f'Original system: {system}')  
        return system
           
    def DefineAdjacents(self, item, PipSystem):
        x = 0
        y = 1
        adj_relative = {"U": [0,1], "D":[0,-1], "R":[1,0], "L":[-1,0]}
        adjacents = []
        for position in adj_relative:
            adj_relative[position] = [item[1][x]+adj_relative[position][0], item[1][y]+ adj_relative[position][1]]
            [adjacents.append([position, item2[0]]) for item2 in PipSystem if item2[1] == adj_relative[position] ]

        return adjacents
       
    #run in Solution()
    def ActiveAdjacents(self, PipSystem):        
        #relative coordinates are [U,L,R,D]
        pipe_act = {"═" : [0,1,1,0], "║" : [1,0,0,1], "╔" : [0,0,1,1], "╗" : [0,1,0,1], "╚" : [1,0,1,0], "╝" : [1,1,0,0], "╠" : [1,0,1,1], "╣" : [1,1,0,1], "╦" : [0,1,1,1], "╩" : [1,1,1,0]}
        adj_act = {"U":[1,0,0,0], "L":[0,1,0,0], "R":[0,0,1,0], "D":[0,0,0,1]}
        system_adj = []
        #find active adjacents
        for item in PipSystem:
            adjacents = self.DefineAdjacents(item, PipSystem)
            item_adj = copy.deepcopy(item)
            for adjacent in adjacents:
                item_adj.append(adjacent)
            system_adj.append(item_adj)
              
            for adjacent in item_adj[2:]:   
                #define the activation array for relative positions: U,L,R,D
                activation_relative = adj_act[adjacent[0]]
                #define the activation array for the item itself 
                if item_adj[0] in self.mean_points:
                    activation_pipe = [1,1,1,1]
                elif item_adj[0] in pipe_act.keys():
                    activation_pipe = pipe_act[item_adj[0]]
                #define activation array for the adjacent
                if adjacent[1] in self.mean_points:
                    activation_adjacent = [1,1,1,1]
                elif adjacent[1] in pipe_act:
                    activation_adjacent = pipe_act[adjacent[1]]
                    activation_adjacent = activation_adjacent[::-1]
                    
                active_side_pipe = np.multiply(activation_pipe, activation_relative)  
                #defines the real ACTIVE adjacents that an item has
                active_sides = np.multiply(active_side_pipe, activation_adjacent)
                
                #remove items that dont have any ACTIVE adjacent 
                if np.count_nonzero(active_sides) == 0:
                    system_adj[PipSystem.index(item)].remove(adjacent)
                    
        return system_adj
    
    def Solution(self):
        system = self.GetSystemList()
        system_adj_o = self.ActiveAdjacents(system)
        starting_points = [item[0] for item in system_adj_o if (item[0] in self.mean_points and item[0] != '*')]
        active_points = copy.deepcopy(starting_points)
        for start in starting_points:
            flag = True
            system_active = copy.deepcopy(system)

            while flag:
                problem_system_adj = self.ActiveAdjacents(system_active)
                count_act_adj = 0
               
                for item in problem_system_adj:
                    if len(item) == 2:
                        system_active.remove(item[:2])
                        
                    if len(item) == 3 and item[0] != start and item[0] != '*':
                        system_active.remove(item[:2])
                        count_act_adj = count_act_adj + 1
                if count_act_adj == 0:
                    solution_pipe = []
                    for item in problem_system_adj:
                        solution_pipe.append(item[0:2])
                    if len(solution_pipe) == 1 and solution_pipe[0][0] == "*":
                        active_points.remove(start)

                    flag = False
        print(f'Final Active letters connected to * : {", ".join(active_points)}')
    
test = PipingSystem("system_to_test_1.txt")
test.Solution()

Original system: [['C', [1, 0]], ['╠', [1, 1]], ['╣', [1, 2]], ['═', [2, 1]], ['╚', [3, 0]], ['╝', [3, 1]], ['╔', [3, 2]], ['*', [0, 2]], ['═', [4, 0]], ['═', [4, 2]], ['B', [5, 0]], ['A', [5, 2]], ['D', [10, 10]]]
Final Active letters connected to * : C, A
